# Qualicum Beach GCP Analysis (Google Colab)

This notebook demonstrates how to:
1. Parse ground control points from a KMZ file
2. Download a basemap from OpenStreetMap
3. Visualize GCPs overlaid on the basemap

**Note**: This notebook is designed for Google Colab. Upload your data to Google Drive and mount it to use this notebook.


## Setup: Install Dependencies and Mount Google Drive


In [ ]:
# Install required packages
!pip install numpy>=1.24.0 pillow>=10.0.0 matplotlib>=3.7.0
!pip install rasterio>=1.3.0 requests>=2.31.0 tqdm>=4.66.0

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Setup complete!")


## Upload Package Files

Upload the `qualicum_beach_gcp_analysis` package directory to `/content/` or clone from repository.


In [ ]:
# If you have the package in Google Drive, add it to path
# Or clone from repository if available
import sys
from pathlib import Path

# Update this path to point to your package location
package_path = Path("/content/qualicum_beach_gcp_analysis")
if package_path.exists():
    sys.path.insert(0, str(package_path))
    print(f"Added {package_path} to Python path")
else:
    print("Package not found. Please upload the qualicum_beach_gcp_analysis directory to /content/")
    print("Or install from repository if available")


## Imports and Configuration


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import package (adjust path as needed)
try:
    from qualicum_beach_gcp_analysis import (
        load_gcps_from_kmz,
        download_basemap,
        visualize_gcps_on_basemap,
        export_to_metashape,
        export_to_metashape_csv,
        export_to_metashape_xml,
    )
    from qualicum_beach_gcp_analysis.visualization import calculate_gcp_bbox
    print("Imports successful!")
except ImportError as e:
    print(f"Import error: {e}")
    print("Please ensure the package is in the Python path")


## 1. Load Ground Control Points from KMZ File

**Note**: Update the path below to point to your KMZ file in Google Drive.


In [ ]:
# Path to the KMZ file in Google Drive
# Update this path to match your file location
kmz_path = "/content/drive/MyDrive/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz"

# Alternative: Upload file directly
# from google.colab import files
# uploaded = files.upload()
# kmz_path = list(uploaded.keys())[0]

# Load GCPs
gcps = load_gcps_from_kmz(kmz_path)

print(f"\nLoaded {len(gcps)} ground control points")

# Display first few GCPs
if gcps:
    print("\nFirst few GCPs:")
    for i, gcp in enumerate(gcps[:5]):
        print(f"  {i+1}. {gcp.get('id', 'Unknown')}: ({gcp['lat']:.6f}, {gcp['lon']:.6f})")


## 2. Calculate Bounding Box from GCPs


In [ ]:
# Calculate bounding box with padding
bbox = calculate_gcp_bbox(gcps, padding=0.01)
min_lat, min_lon, max_lat, max_lon = bbox

print(f"Bounding box:")
print(f"  Latitude: {min_lat:.6f} to {max_lat:.6f}")
print(f"  Longitude: {min_lon:.6f} to {max_lon:.6f}")
print(f"  Span: {max_lat - min_lat:.6f}° lat, {max_lon - min_lon:.6f}° lon")

# Calculate H3 cells that overlap this region
try:
    from qualicum_beach_gcp_analysis import bbox_to_h3_cells
    
    # Use resolution 12 (good balance between detail and number of cells)
    h3_cells = bbox_to_h3_cells(bbox, resolution=12)
    
    print(f"\nH3 cells overlapping this region (resolution 12):")
    print(f"  Total cells: {len(h3_cells)}")
    print(f"  First 10 cells: {h3_cells[:10]}")
    if len(h3_cells) > 10:
        print(f"  Last 10 cells: {h3_cells[-10:]}")
except ImportError as e:
    print(f"\n⚠️  Could not calculate H3 cells: {e}")
    print("  Install h3 library with: pip install h3")
except Exception as e:
    print(f"\n⚠️  Error calculating H3 cells: {e}")


## 3. Download Basemap


In [ ]:
# Create output directory
output_dir = Path("/content/outputs")
output_dir.mkdir(exist_ok=True)

# Download basemap (OpenStreetMap)
basemap_path = str(output_dir / "qualicum_beach_basemap.tif")

print("Downloading basemap from OpenStreetMap...")
basemap_path = download_basemap(
    bbox=bbox,
    output_path=basemap_path,
    source="openstreetmap",
    zoom=None  # Auto-calculate zoom level
)

print(f"\nBasemap saved to: {basemap_path}")


## 4. Visualize GCPs on Basemap


In [ ]:
# Create visualization
visualization_path = str(output_dir / "qualicum_beach_gcps_visualization.png")

visualize_gcps_on_basemap(
    gcps=gcps,
    basemap_path=basemap_path,
    output_path=visualization_path,
    title="Qualicum Beach Ground Control Points",
    point_size=100,
    point_color='red',
    show_labels=True
)

print(f"\nVisualization saved to: {visualization_path}")

# Display the image
from IPython.display import Image, display
display(Image(visualization_path))


## 5. Alternative: Use Esri World Imagery Basemap


In [ ]:
# Download Esri World Imagery basemap (satellite imagery)
basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri.tif")

print("Downloading basemap from Esri World Imagery...")
basemap_esri_path = download_basemap(
    bbox=bbox,
    output_path=basemap_esri_path,
    source="esri_world_imagery",
    zoom=None
)

# Visualize with Esri basemap
visualization_esri_path = str(output_dir / "qualicum_beach_gcps_visualization_esri.png")

visualize_gcps_on_basemap(
    gcps=gcps,
    basemap_path=basemap_esri_path,
    output_path=visualization_esri_path,
    title="Qualicum Beach Ground Control Points (Esri World Imagery)",
    point_size=100,
    point_color='yellow',
    show_labels=True
)

print(f"\nVisualization saved to: {visualization_esri_path}")

# Display the image
display(Image(visualization_esri_path))


## 6. Summary Statistics


## 7. Export GCPs for MetaShape


In [ ]:
# Export GCPs to MetaShape format
from qualicum_beach_gcp_analysis import export_to_metashape_csv, export_to_metashape_xml

# Export as CSV (tab-separated) - most common format
metashape_csv_path = export_to_metashape_csv(
    gcps=gcps,
    output_path=str(output_dir / "qualicum_beach_gcps_metashape.txt")
)

# Export as XML marker file (alternative format)
metashape_xml_path = export_to_metashape_xml(
    gcps=gcps,
    output_path=str(output_dir / "qualicum_beach_gcps_metashape.xml")
)

print(f"\n✓ GCPs exported for MetaShape:")
print(f"  CSV format: {metashape_csv_path}")
print(f"  XML format: {metashape_xml_path}")
print(f"\nTo use in MetaShape:")
print(f"  1. Open your project in MetaShape")
print(f"  2. Go to Tools > Import > Import Markers")
print(f"  3. Select the CSV file (qualicum_beach_gcps_metashape.txt)")
print(f"  4. Or use File > Import > Import Markers for XML format")
print(f"\nThese files can be downloaded from the Colab file browser.")


In [ ]:
print("\n=== GCP Summary ===")
print(f"Total GCPs: {len(gcps)}")

if gcps:
    lats = [gcp['lat'] for gcp in gcps]
    lons = [gcp['lon'] for gcp in gcps]
    elevations = [gcp.get('z', 0) for gcp in gcps]
    
    print(f"\nLatitude range: {min(lats):.6f} to {max(lats):.6f}")
    print(f"Longitude range: {min(lons):.6f} to {max(lons):.6f}")
    
    if any(elevations):
        print(f"Elevation range: {min(elevations):.2f} to {max(elevations):.2f} m")
    
    print(f"\nBounding box:")
    print(f"  Min: ({min_lat:.6f}, {min_lon:.6f})")
    print(f"  Max: ({max_lat:.6f}, {max_lon:.6f})")
